In [9]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 207f070e-0769-45e3-aa47-511c9abbbb47, 11, Finished, Available, Finished)

In [10]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")

StatementMeta(, 207f070e-0769-45e3-aa47-511c9abbbb47, 12, Finished, Available, Finished)

In [11]:
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, 207f070e-0769-45e3-aa47-511c9abbbb47, 13, Finished, Available, Finished)

In [12]:
result = model.transform(df)

StatementMeta(, 207f070e-0769-45e3-aa47-511c9abbbb47, 14, Finished, Available, Finished)

In [13]:
from pyspark.sql.functions import col, to_date
sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 207f070e-0769-45e3-aa47-511c9abbbb47, 15, Finished, Available, Finished)

In [14]:
sentiment_df_final = sentiment_df.drop("error", "response")

StatementMeta(, 207f070e-0769-45e3-aa47-511c9abbbb47, 16, Finished, Available, Finished)

In [21]:
from pyspark.sql.functions import to_date

# Convert the 'datePublished' column to a date datatype
sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date("datePublished", "dd-MMM-yyyy"))

StatementMeta(, 207f070e-0769-45e3-aa47-511c9abbbb47, 23, Finished, Available, Finished)

In [25]:

from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    sentiment_df_final.write.mode("overwrite").format('delta').saveAsTable(table_name)

except AnalysisException:
    print('Table already exists')

    df_cleaned_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""   MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view

                    ON source_view.url = target_table.url

                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *
        """)

StatementMeta(, 207f070e-0769-45e3-aa47-511c9abbbb47, 27, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o6116.saveAsTable.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.sql.delta.perf.DeltaOptimizedWriterExec.awaitShuffleMapStage$1(DeltaOptimizedWriterExec.scala:157)
	at org.apache.spark.sql.delta.perf.DeltaOptimizedWriterExec.getShuffleStats(DeltaOptimizedWriterExec.scala:162)
	at org.apache.spark.sql.delta.perf.DeltaOptimizedWriterExec.computeBins(DeltaOptimizedWriterExec.scala:103)
	at org.apache.spark.sql.delta.perf.DeltaOptimizedWriterExec.doExecute(DeltaOptimizedWriterExec.scala:178)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:200)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:251)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	